<a href="https://colab.research.google.com/github/nkrj01/Unit-ops-classification/blob/main/keras_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introduction**
In this notebook, I have created an NLP model to clasify health authority questions into certain themes which can be used to gain important insights to drive important process changes

**Importing libraries**

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import re
import string
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')  # Download the stop words list
from nltk.corpus import stopwords

**Importing RTQ data**

In [ ]:
data = pd.read_csv(r'train.csv')
df_train = data[["Questions", "target"]]
# train-test split
X_train, X_CV, y_train, y_CV = train_test_split(df_train["text_train"],
                                                df_train["target"],
                                                test_size=0.1,
                                                random_state=21,
                                                stratify=df_train["target"])

**Creating a Keras layer to clean and vectorize the text**

In [ ]:
def text_cleaning(text):
  text_tf = tf.convert_to_tensor(text, dtype=tf.string)
  lowercase = tf.strings.lower(text_tf)
  striped_nonenglish = tf.strings.regex_replace(lowercase, '[^ -~]', '')
  stripped_links = tf.strings.regex_replace(striped_nonenglish, r"https?://\S+|www\.\S+", " ")
  stripped_special_char = tf.strings.regex_replace(
        stripped_links, f"[{re.escape(string.punctuation)}]", ""
        )
  clean_text = tf.strings.strip(stripped_special_char) # strip white space
  stop_words = set(stopwords.words('english'))
  stop_word_pattern = '\\b(' + '|'.join(stop_words) + ')\\b'
  main_words = tf.strings.regex_replace(clean_text, stop_word_pattern, '')

  return main_words

max_features = 1000
embedding_dim = 128
sequence_length = 200

vectorize_layer = TextVectorization(
    standardize=text_cleaning,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# fitting the vectorization layer into training data
vectorize_layer.adapt(X_train.tolist() + X_CV.tolist())

# converting to tf object for model input later
X_train = tf.convert_to_tensor(X_train, dtype=tf.string)
X_CV = tf.convert_to_tensor(X_CV, dtype=tf.string)

**Neural network model training without tuning**

In [ ]:
inputs = tf.keras.Input(shape=(1,), dtype=tf.string) # input layer
x = vectorize_layer(inputs) # vectorization
x = layers.Embedding(max_features+1, embedding_dim)(x) # embeddings
x = layers.Dropout(0.5)(x)
x = layers.Conv1D(50, 4, padding="valid", activation="relu", strides=1)(x) # conv layer to detect relations
x = layers.GlobalMaxPooling1D()(x) # pooling to reduce dimensionality
x = layers.Dense(32, activation="relu")(x) # dense layer
output = layers.Dense(8, activation="softmax", name="predictions")(x) # multi-class output layer

model_nn = tf.keras.Model(inputs, output)

# compile
model_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalCrossentropy(),
                       tf.keras.metrics.F1Score()])
# fit
model_nn.fit(
    X_train,
    y_train.to_numpy().reshape(-1, 1).astype(np.float32),
    batch_size = 200,
    epochs = 5,
    validation_data=(X_CV, y_CV.to_numpy().reshape(-1, 1).astype(np.float32))
)

**Hyperparameter tuning using Keras tuner**

In [ ]:
#  wrap the model inside a function
def build_model(hp):
  inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
  x = vectorize_layer(inputs)
  x = layers.Embedding(max_features+1, embedding_dim)(x)
  x = layers.Dropout(0.5)(x)
  # optimzing kernel size and filter numbers
  x = layers.Conv1D(
      hp.Int("filters", min_value=8, max_value=64, step=8),
      hp.Int("kernel size", min_value=2, max_value=6, step=1),
      padding="valid", activation="relu", strides=1)(x)
  x = layers.GlobalMaxPooling1D()(x)
  # optimizing number of units of dense layer
  x = layers.Dense(hp.Int("units", min_value=8, max_value=64, step=8),
                   activation="relu")(x)
  output = layers.Dense(10, activation="softmax", name="predictions")(x)

  model_nn = tf.keras.Model(inputs, output)

  # compile
  model_nn.compile(optimizer=tf.keras.optimizers.Adam(
                hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), # optimizing alpha
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=[tf.keras.metrics.tf.keras.metrics.SparseCategoricalAccuracy(),
                        tf.keras.metrics.F1Score()])

  return model_nn

# setting up the tuner
tuner = keras_tuner.tuners.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=10)

# searching optimal model
tuner.search(
    X_train,
    y_train.to_numpy().reshape(-1, 1).astype(np.float32),
    validation_data=(X_CV, y_CV.to_numpy().reshape(-1, 1).astype(np.float32)))

# retrieving optimal model
model_nn = tuner.get_best_models(num_models=1)[0]